In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [3]:
from bert_score import score
import torch
from transformers import AutoTokenizer, AutoModel
import torch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [11]:
nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

In [9]:
tokenizer(["a", "b"]).encodings[0].tokens

['<s>', 'a', '</s>']

In [13]:
print([tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token])

['<s>', 'return', 'Ġmaximum', 'Ġvalue', '</s>', 'def', 'Ġmax', '(', 'a', ',', 'b', '):', 'Ġif', 'Ġa', '>', 'b', ':', 'Ġreturn', 'Ġa', 'Ġelse', 'Ġreturn', 'Ġb', '</s>']


In [118]:
with torch.no_grad():
    o = model(**tokenizer(["def max(a,b): if a>b: return a else return b", "def min(i,j): if i>j: return i else return j"], return_tensors='pt'))

In [14]:
cos_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [119]:
o.pooler_output.shape

torch.Size([2, 768])

In [80]:
o.pooler_output[0,:].unsqueeze(0).shape

torch.Size([1, 768])

In [122]:
df[df['language_name'] == 'Python'].sample()['code']

43448    import sys\nimport pygame\n \npygame.init()\n ...
Name: code, dtype: object

In [129]:
df[df['language_name'] == 'Python']['code'].apply(len).describe()

count     1136.000000
mean      1081.649648
std       1916.171263
min          3.000000
25%        245.750000
50%        640.500000
75%       1300.250000
max      46867.000000
Name: code, dtype: float64

In [21]:
tokenizer("\t").encodings[0].tokens

['<s>', 'ĉ', '</s>']

In [107]:
print(c)

>>> def duration(seconds):
	t= []
	for dm in (60, 60, 24, 7):
		seconds, m = divmod(seconds, dm)
		t.append(m)
	t.append(seconds)
	return ', '.join('%d %s' % (num, unit)
			 for num, unit in zip(t[::-1], 'wk d hr min sec'.split())
			 if num)
 
>>> for seconds in [7259, 86400, 6000000]:
	print("%7d sec = %s" % (seconds, duration(seconds)))
 
 
   7259 sec = 2 hr, 59 sec
  86400 sec = 1 d
6000000 sec = 9 wk, 6 d, 10 hr, 40 min
>>> 


In [120]:
cos_sim(o.pooler_output[0,:].unsqueeze(0), o.pooler_output[1,:].unsqueeze(0))

tensor([0.9992])

In [30]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def max(i,j): if i>j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

CPU times: user 1.33 s, sys: 328 ms, total: 1.66 s
Wall time: 5.72 s


(tensor([0.8903]), tensor([0.8903]), tensor([0.8903]))

In [32]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def min(i,j): if i<j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

CPU times: user 1.28 s, sys: 186 ms, total: 1.47 s
Wall time: 5.55 s


(tensor([0.8669]), tensor([0.8669]), tensor([0.8669]))

In [26]:
print(f"{Precision.numpy()=}, {Recall=}, {F1=}")

Precision.numpy()=array([0.8903439], dtype=float32), Recall=tensor([0.8903]), F1=tensor([0.8903])


In [157]:
Precision.numpy()

array([0.9680341], dtype=float32)

In [14]:
import datasets

In [16]:
from datasets import load_dataset

dataset_py = load_dataset("code_x_glue_cc_code_completion_line", "python")
datasert_j = load_dataset("code_x_glue_cc_code_completion_line", "java")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset code_x_glue_cc_code_completion_line downloaded and prepared to /tf/data/cache/HF/datasets/code_x_glue_cc_code_completion_line/python/0.0.0/4ce8a216b87c5b130aad675f2bbf3612cc1f7fa5dbdc2fcb9c412765cf7830a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset code_x_glue_cc_code_completion_line downloaded and prepared to /tf/data/cache/HF/datasets/code_x_glue_cc_code_completion_line/java/0.0.0/4ce8a216b87c5b130aad675f2bbf3612cc1f7fa5dbdc2fcb9c412765cf7830a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
bloom = AutoModel.from_pretrained('bigscience/bloom-350m')

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [24]:
print(dataset_py['train'][0]['input'].replace('<EOL>', '\n'))

<s> from __future__ import absolute_import 
 import weakref 
 import operator 
 from . compat import threading , itertools_filterfalse 
 from . import py2k 
 import types 
 EMPTY_SET = frozenset ( ) 
 class KeyedTuple ( tuple ) : 
 def __new__ ( cls , vals , labels = None ) : 
 t = tuple . __new__ ( cls , vals ) 
 t . _labels = [ ] 
 if labels : 
 t . __dict__ . update ( zip ( labels , vals ) ) 
 t . _labels = labels 
 return t 
 def keys ( self ) : 
 return [ l for l in self . _labels if l is not None ] 
 @ property 
 def _fields ( self ) : 
 return tuple ( self . keys ( ) ) 
 def _asdict ( self ) : 
 return dict ( ( key , self . __dict__ [ key ] ) for key in self . keys ( ) ) 
 class ImmutableContainer ( object ) : 
 def _immutable ( self , * arg , ** kw ) : 
 raise TypeError ( "" % self . __class__ . __name__ ) 
 __delitem__ = __setitem__ = __setattr__ = _immutable 
 class immutabledict ( ImmutableContainer , dict ) : 
 clear = pop = popitem = setdefault = update = ImmutableContaine

In [4]:
import torch

In [6]:
torch.cuda.current_device()

0

In [161]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, StoppingCriteria, StoppingCriteriaList
from transformers import pipeline

In [8]:
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")

Downloading:   0%|          | 0.00/259 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/821k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/903 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [163]:
class EndOfFunctionCriteria(StoppingCriteria):
    """Custom `StoppingCriteria` which checks if all generated functions in the batch are completed."""

    def __init__(self, start_length, eof_strings, tokenizer):
        self.start_length = start_length
        self.eof_strings = eof_strings
        self.tokenizer = tokenizer

    def __call__(self, input_ids, scores, **kwargs):
        """Returns true if all generated sequences contain any of the end-of-function strings."""
        decoded_generations = self.tokenizer.batch_decode(input_ids[:, self.start_length :])
        done = []
        for decoded_generation in decoded_generations:
            done.append(any([stop_string in decoded_generation for stop_string in self.eof_strings]))
        return all(done)

In [171]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generated_text = pipe("def hello_world():\n", stopping_criteria=StoppingCriteriaList([EndOfFunctionCriteria(0, [' def'], tokenizer)]), typical_p=0.9 ,max_new_tokens=50)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
#        '''
#        '''
#        pass

#    def


In [21]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generated_text = pipe("def hello_world():\n", typical_p=0.9 ,max_new_tokens=50)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
#        '''
#        '''
#        pass

#    def get_output(self, timeout=None):
#        '''
#        '''
#        return self.output


if __name__ == '__main


In [25]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "def hello_world():"
generated_text = pipe(prompt, typical_p=0.9 ,max_new_tokens=50, device=0)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
    print('Hello!')

if __name__ == '__main__':
    hello_world()
# coding=utf-8
# --------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT


In [156]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "def hello_world():\n"
generated_text = pipe(prompt, do_sample=True, typical_p=0.9 ,max_new_tokens=50, device=0)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
#        '''
#        '''
#        pass

#    def get_output(self, timeout=None):
#        '''
#        '''
#        return self.output


if __name__ == '__main


In [160]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "def hello_world():\n"
generated_text = pipe(prompt, do_sample=True, typical_p=0.7 ,max_new_tokens=50, device=0)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
	print('Hello, world!')
	print('Hello, world!')
	print('Hello, world!')

def hello_world2():
	print('Hello, world!')
	print('Hello, world!')
	print


In [37]:
set_seed(42)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "def hello_world():"
generated_text = pipe(prompt,  temperature=0.6, do_sample=True, top_p=0.95, device=0)[0]['generated_text']
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
    return "Hello World!"

def hello_world_with_default():
    return "Hello World!"

def hello_world_with_default_with_default():
    return "Hello World!"



In [30]:
generated_text[len(prompt):].split('\n')

['',
 "    print('Hello!')",
 '',
 "if __name__ == '__main__':",
 '    hello_world()',
 '# coding=utf-8',
 '# --------------------------------------------------------------------------',
 '# Copyright (c) Microsoft Corporation. All rights reserved.',
 '# Licensed under the MIT']

In [42]:
human_eval = load_dataset("openai_humaneval", split='test')

Reusing dataset openai_humaneval (/tf/data/cache/HF/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


In [46]:
human_eval[0]['prompt']

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

In [49]:
human_eval.column_names

['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point']

In [67]:
human_eval[10]['canonical_solution']

"    if not string:\n        return ''\n\n    beginning_of_suffix = 0\n\n    while not is_palindrome(string[beginning_of_suffix:]):\n        beginning_of_suffix += 1\n\n    return string + string[:beginning_of_suffix][::-1]\n"

In [71]:
df = human_eval.to_pandas()

In [129]:
human_eval.filter(lambda x: len(x['canonical_solution'].split('\n'))<3)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
    num_rows: 37
})

In [132]:
from datasets import load_dataset
from evaluate import load

In [128]:
len(df[df['canonical_solution'].str.split('\n').apply(len) == 3])

9

In [82]:
import pandas as pd

In [133]:
human_eval = load_dataset("openai_humaneval")
code_eval_metric = load("code_eval")

Reusing dataset openai_humaneval (/tf/data/cache/HF/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


  0%|          | 0/1 [00:00<?, ?it/s]

In [135]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [154]:
%%time
test_cases = ["assert add(2,3)==5"]
candidates = [["def add(a,b): return a*b", "def add(a, b):\n    return a+b"]]
pass_at_k, results = code_eval_metric.compute(references=test_cases, predictions=candidates, k=[1, 2])
print(pass_at_k)

{'pass@1': 0.5, 'pass@2': 1.0}
CPU times: user 24 ms, sys: 243 ms, total: 267 ms
Wall time: 467 ms


In [153]:
print("def add(a, b):\nreturn a+b")

def add(a, b):
return a+b


In [139]:
human_eval['test']

Dataset({
    features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
    num_rows: 164
})

In [84]:
rosetta = pd.read_json('../code_datasets/code_crawl.jsonl', lines=True)

In [85]:
rosetta

,task_url,task_name,task_description,language_url,language_name,code
0,http://rosettacode.org/wiki/Ascending_primes,Ascending primes,Generate and show all primes with strictly asc...,#ALGOL_68,ALGOL 68,BEGIN # find all primes with strictly increasi...
1,http://rosettacode.org/wiki/Ascending_primes,Ascending primes,Generate and show all primes with strictly asc...,#Arturo,Arturo,ascending?: function [x][\n initial: digits...
2,http://rosettacode.org/wiki/Ascending_primes,Ascending primes,Generate and show all primes with strictly asc...,#AWK,AWK,\n# syntax: GAWK -f ASCENDING_PRIMES.AWK\nBEG...
3,http://rosettacode.org/wiki/Ascending_primes,Ascending primes,Generate and show all primes with strictly asc...,#F.23,F#,\n// Ascending primes. Nigel Galloway: April ...
4,http://rosettacode.org/wiki/Ascending_primes,Ascending primes,Generate and show all primes with strictly asc...,#Factor,Factor,USING: grouping math math.combinatorics math.f...
...,...,...,...,...,...,...
79008,http://rosettacode.org/wiki/A%2BB,A+B,A+B ─── a classic problem in programming con...,#zkl,zkl,"do(2){ask(""A B: "").split("" "").filter().sum().p..."
79009,http://rosettacode.org/wiki/A%2BB,A+B,A+B ─── a classic problem in programming con...,#Zoea,Zoea,program: a_plus_b\n input: '7 11' \n output:...
79010,http://rosettacode.org/wiki/A%2BB,A+B,A+B ─── a classic problem in programming con...,#Zoea_Visual,Zoea Visual,"\nmodule ABProblem;\nvar\n\ta,b: integer;\nbe..."
79011,http://rosettacode.org/wiki/A%2BB,A+B,A+B ─── a classic problem in programming con...,#zonnon,zonnon,"\nmodule ABProblem;\nvar\n\ta,b: integer;\nbe..."


In [87]:
from datasets import Dataset

In [88]:
d = Dataset.from_pandas(rosetta)

In [90]:
from huggingface_hub import notebook_login

In [91]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [92]:
d.push_to_hub('rosetta-code')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


In [93]:
rosetta = load_dataset('cakiki/rosetta-code', split='train')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Using custom data configuration cakiki--rosetta-code-495596fe3afff1d5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /tf/data/cache/HF/datasets/cakiki___parquet/cakiki--rosetta-code-495596fe3afff1d5/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


In [96]:
r = rosetta.to_pandas()

Phix      1181
Julia     1179
Wren      1177
Go        1172
Raku      1168
Perl      1147
Python    1139
Nim       1137
J         1038
C         1024
Name: language_name, dtype: int64

In [102]:
rosetta_python = rosetta.filter(lambda x: x['language_name']=='Python')

Loading cached processed dataset at /tf/data/cache/HF/datasets/cakiki___parquet/cakiki--rosetta-code-495596fe3afff1d5/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-6a00c7086ca98b21.arrow


In [119]:
rosetta_python.filter(lambda x: len(x['code'].split('\n')) <= 3 )[:]

  0%|          | 0/2 [00:00<?, ?ba/s]

{'task_url': ['http://rosettacode.org/wiki/Array_length',
  'http://rosettacode.org/wiki/Arbitrary-precision_integers_(included)',
  'http://rosettacode.org/wiki/Write_entire_file',
  'http://rosettacode.org/wiki/Walk_a_directory/Non-recursively',
  'http://rosettacode.org/wiki/Variadic_function',
  'http://rosettacode.org/wiki/URL_encoding',
  'http://rosettacode.org/wiki/User_input/Text',
  'http://rosettacode.org/wiki/Tokenize_a_string',
  'http://rosettacode.org/wiki/Terminal_control/Ringing_the_terminal_bell',
  'http://rosettacode.org/wiki/Terminal_control/Hiding_the_cursor',
  'http://rosettacode.org/wiki/Terminal_control/Inverse_video',
  'http://rosettacode.org/wiki/Terminal_control/Display_an_extended_character',
  'http://rosettacode.org/wiki/Terminal_control/Cursor_positioning',
  'http://rosettacode.org/wiki/Terminal_control/Clear_the_screen',
  'http://rosettacode.org/wiki/System_time',
  'http://rosettacode.org/wiki/Strip_control_codes_and_extended_characters_from_a_stri

In [116]:
r[r['language_name'] == 'Python']['code'].str.split('\n').apply(len).value_counts().sort_index()[:10]

1     24
2     40
3     31
4     33
5     40
6     28
7     29
8     33
9     22
10    20
Name: code, dtype: int64